<a href="https://colab.research.google.com/github/taugroup/2026-IEEE-MOVE-Truck-Data-Science-Challenge/blob/main/Round%201/Analysis_Selected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
#import folium
import json
import seaborn as sns
from datetime import datetime

In [ ]:
# Step 1: Download the file from the new Dropbox URL
!wget "https://www.dropbox.com/scl/fi/mam4v02g9fdg4igmbag1c/router_data.zip?rlkey=dhvdrik7wqfhyqeuzlslb5zns&st=u40ntej2&dl=0" -O router_data.zip

# Step 2: Unzip the downloaded file
!unzip router_data.zip


In [ ]:
DATA_PATH = "./20240902"
print(os.listdir(DATA_PATH))

In [ ]:
import pandas as pd
import glob
import json

def load_json_files(folder_path):
    json_files = glob.glob(f"{folder_path}/**/*.json", recursive=True)
    dataframes = []

    for file in json_files:
        print(f"Processing: {file}")
        try:
            with open(file, 'r') as f:
                data = json.load(f)
            df = pd.json_normalize(data)
            dataframes.append(df)
        except json.JSONDecodeError:
            print(f"Error: Could not parse JSON in {file}")

    return pd.concat(dataframes, ignore_index=True)

# Usage

combined_df = load_json_files(DATA_PATH)

In [ ]:
selected_file = os.path.join(DATA_PATH, 'status.pickle')
combined_df.to_pickle(selected_file)
df = pd.read_pickle(selected_file)
df

In [ ]:
df.columns

In [ ]:
import numpy as np

def calculate_bearing(pointA, pointB):
    lat1, lon1 = np.radians(pointA)
    lat2, lon2 = np.radians(pointB)
    dlon = lon2 - lon1
    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)
    initial_bearing = np.arctan2(x, y)
    initial_bearing = np.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360
    return compass_bearing

In [ ]:
import folium
from folium import plugins

# Extract relevant GPS columns from 'selected' data
gps_columns = ['timestamp',
    'gps_fix.latitude.degree',
    'gps_fix.latitude.minute',
    'gps_fix.latitude.second',
    'gps_fix.longitude.degree',
    'gps_fix.longitude.minute',
    'gps_fix.longitude.second'
]

# Assuming your DataFrame loaded from 'selected' data is also named 'df'
if all(col in df.columns for col in gps_columns):
    gps_df = df[gps_columns].copy()

    # Function to convert degrees, minutes, seconds to decimal degrees
    def dms_to_decimal(degree, minute, second):
        return float(degree) + (float(minute) / 60) + (float(second) / 3600)

    # Convert latitude and longitude to decimal degrees
    gps_df['latitude'] = gps_df.apply(lambda row: dms_to_decimal(
        row['gps_fix.latitude.degree'],
        row['gps_fix.latitude.minute'],
        row['gps_fix.latitude.second']
    ), axis=1)

    gps_df['longitude'] = gps_df.apply(lambda row: dms_to_decimal(
        row['gps_fix.longitude.degree'],
        row['gps_fix.longitude.minute'],
        row['gps_fix.longitude.second']
    ), axis=1)

    center_lat = gps_df['latitude'].mean()
    center_lon = gps_df['longitude'].mean()

    m = folium.Map(location=[center_lat, center_lon], zoom_start=7)

    # Add markers for each GPS coordinate
    for idx, row in gps_df.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"Time: {row['timestamp']}",
            tooltip=f"Lat: {row['latitude']:.6f}, Lon: {row['longitude']:.6f}"
        ).add_to(m)

    # Add a PolyLine to show the movement path
    coordinates = gps_df[['latitude', 'longitude']].values.tolist()
    # folium.PolyLine(coordinates, color="red", weight=2.5, opacity=1).add_to(m)

    # Add AntPath to show movement direction
    plugins.AntPath(
        locations=coordinates,
        color='blue',
        weight=2,
        opacity=0.7
    ).add_to(m)

    # Add markers for start and end points
    folium.Marker(
        coordinates[0],
        popup=f"Start: {gps_df['timestamp'].iloc[0]}",
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)

    folium.Marker(
        coordinates[-1],
        popup=f"End: {gps_df['timestamp'].iloc[-1]}",
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(m)

    # Add direction arrows (optional)
    # for i in range(len(gps_df) - 1):
    #     folium.PolyLine(
    #         locations=[coordinates[i], coordinates[i+1]],
    #         color='red',
    #         weight=2,
    #         opacity=0.8,
    #         arrow_style='->',
    #         arrow_size=6
    #     ).add_to(m)

    # Display the map (in a notebook environment)
    m
else:
    print(f"Error: Not all required columns ({gps_columns}) are present in the DataFrame.")

In [ ]:
directions = []
for i in range(len(gps_df) - 1):
    pointA = (gps_df.iloc[i]['latitude'], gps_df.iloc[i]['longitude'])
    pointB = (gps_df.iloc[i + 1]['latitude'], gps_df.iloc[i + 1]['longitude'])
    directions.append(calculate_bearing(pointA, pointB))

directions.append(None)  # No direction for the last point
gps_df['direction'] = directions

In [ ]:
m.save("map_selected.html")
m